In [17]:
import nltk            # natural language tool kit
import numpy as np     # support for large data structures
import pandas as pd    # data structure support
import re

from importlib import reload 

# POS tagging functions
import pos3

# Load the Penn Treebank Corpus which will serve as our training set.
corpus = nltk.corpus.ptb

In [208]:
train = pd.read_csv('Data/train_split.csv')
train_lite = pd.read_csv('Data/train_lite.csv')
dev = pd.read_csv('Data/dev_split.csv')
del train["Unnamed: 0"]
del dev["Unnamed: 0"]
del train_lite["Unnamed: 0"]
del train_lite["Unnamed: 0.1"]

,id,qid1,qid2,question1,question2,is_duplicate
0,57674,5725,38477,How can I get a list of my Gmail accounts?,How do I find my list of GMail addresses?,1
1,185568,283254,283255,What are the symptoms of child abuse?,What are signs of child abuse?,1
2,359111,400936,488715,What are some examples of selfishness?,Do we all occasionally tend to emotional masoc...,0
3,204216,281934,307050,Is Qnet a scam?,Where is the registered office of Qnet in Mumbai?,0
4,5649,11103,11104,What do Americans think about Donald Trump?,What do you think about Donald Trump pick?,1


In [124]:
reload(pos3)
hmm = pos3.HMM()
for sentence in corpus.tagged_sents():
    hmm.update_counts(sentence)
hmm.compute_logprobs()

In [200]:
def tags_as_string(str):
    p = re.compile("[^A-Za-z ]")
    s = p.sub("", str).split()+["?"]
    r = " ".join(hmm.viterbi(s))
    return r

In [196]:
train_lite["question1_pos"] = train_lite["question1"].apply(tags_as_string)
train_lite["question2_pos"] = train_lite["question2"].apply(tags_as_string)
train_lite.to_pickle("Data/train_lite.df.pkl.gz")

In [ ]:
dev["question1_pos"] = dev["question1"].apply(tags_as_string)
dev["question2_pos"] = dev["question2"].apply(tags_as_string)
dev.to_pickle("Data/train_lite.df.pkl.gz")

In [ ]:
train["question1_pos"] = train["question1"].apply(tags_as_string)
train["question2_pos"] = train["question2"].apply(tags_as_string)
train.to_pickle("Data/train_lite.df.pkl.gz")